In [2]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd

## Dataset Initialization

In [3]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

text = res.text
text[:100]

'pair_ID\tsentence_A\tsentence_B\trelatedness_score\tentailment_judgment\n1\tA group of kids is playing in '

In [4]:
data = pd.read_csv(StringIO(text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [6]:
sentences = data['sentence_A'].tolist()
sentences[:5], len(sentences)

(['A group of kids is playing in a yard and an old man is standing in the background',
  'A group of children is playing in the house and there is no man standing in the background',
  'The young boys are playing outdoors and the man is smiling nearby',
  'The kids are playing outdoors near a man with a smile',
  'The young boys are playing outdoors and the man is smiling nearby'],
 4500)

In [7]:
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)
len(set(sentences)), len(sentence_b)

(4802, 4500)

In [8]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [11]:
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t', header=None, on_bad_lines='skip')
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

In [12]:
len(set(sentences))

14505

Before converting to our sentence embeddings, we will save to text file as backup.

In [14]:
# remove duplicates and NaN
sentences = [
    sentence.replace('\n', '') for sentence in list(set(sentences)) if type(sentence) is str
    ]
len(set(sentences))

14504

In [15]:
with open('sentences.txt', 'w') as fp:
    fp.write('\n'.join(sentences))

Now we have 14.5K unique sentences, a much better size. We'll go ahead and build the sentence embeddings (this can take some time, feel free to download the embeddings from here). TODO add link

In [16]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

/opt/anaconda3/envs/ai-playground/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(14504, 768)

In [18]:
with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
    np.save(fp, sentence_embeddings[0:256])

In [19]:
# saving data
split = 256
file_count = 0
for i in range(0, sentence_embeddings.shape[0], split):
    end = i + split
    if end > sentence_embeddings.shape[0] + 1:
        end = sentence_embeddings.shape[0] + 1
    file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
    with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
        np.save(fp, sentence_embeddings[i:end, :])
    print(f"embeddings_{file_count}.npy | {i} -> {end}")
    file_count = int(file_count) + 1

embeddings_0.npy | 0 -> 256
embeddings_1.npy | 256 -> 512
embeddings_2.npy | 512 -> 768
embeddings_3.npy | 768 -> 1024
embeddings_4.npy | 1024 -> 1280
embeddings_5.npy | 1280 -> 1536
embeddings_6.npy | 1536 -> 1792
embeddings_7.npy | 1792 -> 2048
embeddings_8.npy | 2048 -> 2304
embeddings_9.npy | 2304 -> 2560
embeddings_10.npy | 2560 -> 2816
embeddings_11.npy | 2816 -> 3072
embeddings_12.npy | 3072 -> 3328
embeddings_13.npy | 3328 -> 3584
embeddings_14.npy | 3584 -> 3840
embeddings_15.npy | 3840 -> 4096
embeddings_16.npy | 4096 -> 4352
embeddings_17.npy | 4352 -> 4608
embeddings_18.npy | 4608 -> 4864
embeddings_19.npy | 4864 -> 5120
embeddings_20.npy | 5120 -> 5376
embeddings_21.npy | 5376 -> 5632
embeddings_22.npy | 5632 -> 5888
embeddings_23.npy | 5888 -> 6144
embeddings_24.npy | 6144 -> 6400
embeddings_25.npy | 6400 -> 6656
embeddings_26.npy | 6656 -> 6912
embeddings_27.npy | 6912 -> 7168
embeddings_28.npy | 7168 -> 7424
embeddings_29.npy | 7424 -> 7680
embeddings_30.npy | 7680 -> 7

We setup our FAISS database dimensionality (number of dimensions per vector) based on these vectors.

In [20]:
d = sentence_embeddings.shape[1]
d

768

## Flat L2 Index
We initialize the flat L2 distance index IndexFlatL2, all we need is the specify the vector dimensionality - which in this case is d == 768 (to align with the sentence-BERT model output embeddings of size 768).

In [21]:
index = faiss.IndexFlatL2(d)

Often, we will use indexes that require us to train them on our data before being used (if we are grouping or transforming the data in any way). IndexFlatL2 however, is a simple operation and only requires that we calculate distances between vectors when we introduce our query vector xq during search. So, in this case, no training is required - which we can confirm by checking the is_trained attribute.

In [22]:
index.is_trained

True

Okay so once we're happy that our index is prepared, we then add new vectors using the add method.

In [23]:
index.add(sentence_embeddings)

In [24]:
index.ntotal

14504

Then search given a query xq and number of nearest neigbors to return k.

In [25]:
k = 4
xq = model.encode(["Someone sprints with a football"])

In [26]:
%%time
D, I = index.search(xq, k)  # search
print(I)  # k-nearest neigbors of the query vector | nprobe == 1: 6495 26392 61709 49932 | nprobe == 10: 36245  6495 57489  8705

[[7090 2963 7946 7810]]
CPU times: user 2.93 ms, sys: 2.89 ms, total: 5.82 ms
Wall time: 5.06 ms


In [31]:
[f'{i}: {sentences[i]}' for i in I[0]]

['7090: A group of football players is running in the field',
 '2963: A group of people playing football is running in the field',
 '7946: Two groups of people are playing football',
 '7810: A person playing football is running past an official carrying a football']

In [33]:
sentences[7090]

'A group of football players is running in the field'

Clearly we have some good matches, everything returned includes people running with a football, or on the context of a football match. Now, if we'd rather extract the numerical vectors from FAISS, we can do that too.

In [34]:
vecs = np.zeros((k, d))
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [35]:
vecs.shape

(4, 768)

In [36]:
vecs[0][:100]

array([ 0.01627056,  0.22325902, -0.15037405, -0.30747285, -0.27122435,
       -0.1059322 , -0.06460907,  0.04738186, -0.73349047, -0.37657705,
       -0.76762813,  0.16902864,  0.53107703,  0.51176685,  1.14415872,
       -0.08562887, -0.67240077, -0.9663704 ,  0.02545477, -0.2155983 ,
       -1.25656593, -0.82982141, -0.09825017, -0.21850841,  0.5061022 ,
        0.10527914,  0.50396866,  0.65242904, -1.39458704,  0.65847462,
       -0.21525347, -0.22487469,  0.81818348,  0.08464274, -0.76141673,
       -0.28928289, -0.09825772, -0.73046225,  0.07855783, -0.84354621,
       -0.59242058,  0.77471304, -1.20920527, -0.22757967, -1.30733633,
       -0.23081465, -1.31322539,  0.01629083, -0.97285444,  0.19308178,
        0.47424588,  1.18920887, -1.96741331, -0.70061111, -0.29638639,
        0.60533714,  0.62407392, -0.70340365, -0.86754227,  0.17673175,
       -0.19170491, -0.02951992,  0.22623569, -0.16695412, -0.80402535,
       -0.45918909,  0.69675493, -0.24928212, -1.01478708, -0.92


## Adding Partitioning to the Index
FAISS allows us to add an additional step to optimize our search efficiency using a variety of different methods. A popular approach is to partition the index into Voronoi cells. Using this method we would take our query vector xq, identify the cell it belongs to, and then use our IndexFlatL2 to search between the query vector xq and all indexed vectors belonging to that cell. We can also include vectors from other nearby cells too.

We initialize our new partitioned index by first adding our previous IndexFlatL2 operation as a quantization step (another step in the search process), and feeding this into the new IndexIVFFlat operation like so:

In [58]:
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

Here we've added a new parameter nlist. We use nlist to define how many partitions we'd like our index to have.

When we built the previous, IndexFlatL2-only index, we noted that no training was required as no grouping/transformation was required to build that index. Now that we've added partitioning using IndexIVFFlat, this is no longer the case. Let's take a look at the is_trained attribute.

In [59]:
index.is_trained

False

In [60]:
index.train(sentence_embeddings)
index.is_trained

True

In [61]:
index.add(sentence_embeddings)
index.ntotal

14504

Let's search again using the same indexed sentence embeddings and the same query xq.

In [62]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[2963 7946 7810 5174]]
CPU times: user 365 μs, sys: 134 μs, total: 499 μs
Wall time: 447 μs


We can increase the number of nearby cells to search too with nprobe.

In [63]:
index.nprobe = 10  # increase the number of nearby cells to search

In [64]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[7090 2963 7946 7810]]
CPU times: user 1.35 ms, sys: 1.41 ms, total: 2.76 ms
Wall time: 1.63 ms


Rerunning can change the time to to the search. Let's do the search again and confirm

In [65]:
%%time
D, I = index.search(xq, k)
print(I)

[[7090 2963 7946 7810]]
CPU times: user 1.15 ms, sys: 791 μs, total: 1.94 ms
Wall time: 910 μs


For IVF (and IMI) indexes, before attempting to use the reconstruct method, we need to call the make_direct_map method - otherwise we will return a RunetimeError.

In [66]:
index.reconstruct(I[0][0].item())

RuntimeError: Error in idx_t faiss::DirectMap::get(idx_t) const at /Users/runner/work/faiss-wheels/faiss-wheels/faiss/faiss/invlists/DirectMap.cpp:83: direct map not initialized

With make_direct_map:

In [67]:
index.make_direct_map()

In [68]:
index.reconstruct(I[0][0].item())[:100]

array([ 0.01627056,  0.22325902, -0.15037405, -0.30747285, -0.27122435,
       -0.1059322 , -0.06460907,  0.04738186, -0.73349047, -0.37657705,
       -0.76762813,  0.16902864,  0.531077  ,  0.51176685,  1.1441587 ,
       -0.08562887, -0.6724008 , -0.9663704 ,  0.02545477, -0.2155983 ,
       -1.2565659 , -0.8298214 , -0.09825017, -0.21850841,  0.5061022 ,
        0.10527914,  0.50396866,  0.65242904, -1.394587  ,  0.6584746 ,
       -0.21525347, -0.22487469,  0.8181835 ,  0.08464274, -0.76141673,
       -0.2892829 , -0.09825772, -0.73046225,  0.07855783, -0.8435462 ,
       -0.5924206 ,  0.77471304, -1.2092053 , -0.22757967, -1.3073363 ,
       -0.23081465, -1.3132254 ,  0.01629083, -0.97285444,  0.19308178,
        0.47424588,  1.1892089 , -1.9674133 , -0.7006111 , -0.2963864 ,
        0.60533714,  0.6240739 , -0.70340365, -0.86754227,  0.17673175,
       -0.19170491, -0.02951992,  0.22623569, -0.16695412, -0.80402535,
       -0.4591891 ,  0.69675493, -0.24928212, -1.0147871 , -0.92

We've now significantly reduced the search time, what can we do next?

## Quantization
Well, when storing these vectors we're storing the full (eg Flat) vector. Now in very big datasets this can quickly become a problem. Typically, we look at big datasets, and when working with large dataset we will find that storing the full vectors consumes too much space.

Fortunately, FAISS comes with the ability to compress our vectors using transformations based on Product Quantization (PQ). But, what is PQ? Well, we can view it as an additional approximation step similar to our use of IVF, which allowed us to approximate by reducing the scope of our search. PQ is slightly different however, and approximates the distance (or similarity) calculation instead.

PQ explanation TODO REMOVE

PQ achieves this approximated distance operation by compressing the vectors themselves. This consists of a few steps.

  1. We split the every original vector into several subvectors.

  2. For each set of subvectors, we perform a clustering operation, creating many centroids for each subvector set.

  3. In our vector of subvectors, we replace each subvector with the ID of it's nearest centroid.

In [69]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

In [70]:
index.is_trained

False

In [71]:
index.train(sentence_embeddings)
index.add(sentence_embeddings)

In [72]:
index.nprobe = 10

In [73]:
%%time
D, I = index.search(xq, k)
print(I)

[[ 2963  3324 14163  7295]]
CPU times: user 552 μs, sys: 851 μs, total: 1.4 ms
Wall time: 787 μs


Former list returned: [[7090 2963 7946 7810]]